In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [93]:
import psycopg2

In [94]:
from io import StringIO

In [95]:
from sqlalchemy import create_engine

In [96]:
import pandas as pd
import time
import numpy as np

In [97]:
##连接数据库
con = psycopg2.connect(dbname="cs3072", user="checker",
        password="123456", host="localhost", port="5432")

In [98]:
##创建cur以访问数据库
cur=con.cursor()

In [38]:
big_table=pd.read_csv("contract_info.csv")
big_table

,contract number,client enterprise,supply center,country,city,industry,product code,product name,product model,unit price,...,contract date,estimated delivery date,lodgement date,director,salesman,salesman number,gender,age,mobile phone,Unnamed: 20
0,CSE0000000,Pinduoduo,Eastern China,China,Shanghai,Internet,T67P542,Tv Base,TvBaseR1,680,...,2018-02-02,2018-04-03,2018-03-13,Xu Zhuyu,Jiang Qizhen,12111414,Female,19,15648228450,NaN
1,CSE0000000,Pinduoduo,Eastern China,China,Shanghai,Internet,E6N2308,Electronic Dictionary,ElectronicDictionary67,970,...,2018-02-02,2018-03-12,2018-02-28,Xu Zhuyu,Lv Qiufeng,12110809,Female,19,15088530748,NaN
2,CSE0000000,Pinduoduo,Eastern China,China,Shanghai,Internet,E9G8372,Exhaust Fan,ExhaustFanD8,320,...,2018-02-02,2018-02-05,2018-03-02,Xu Zhuyu,Zheng Runyi,11812628,Male,22,15613461351,NaN
3,CSE0000000,Pinduoduo,Eastern China,China,Shanghai,Internet,M421X86,Multifunctional,MultifunctionalT4,90,...,2018-02-02,2018-02-23,2018-03-03,Xu Zhuyu,Dou Youqing,11612519,Female,24,18695976535,NaN
4,CSE0000000,Pinduoduo,Eastern China,China,Shanghai,Internet,SE17564,Server Barebones,ServerBarebonesH4,70,...,2018-02-02,2018-02-19,2018-03-26,Xu Zhuyu,Jiang Qizhen,12111414,Female,19,15648228450,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,CSE0004999,State Power Corporation,Northern China,China,Beijing,Electricity,CC50327,Central Water Heater,CentralWaterHeaterV6,240,...,2010-03-04,2010-03-16,2010-03-28,Kong Yibo,Han Hanwu,12110927,Male,19,18637769897,NaN
49996,CSE0004999,State Power Corporation,Northern China,China,Beijing,Electricity,C9520Y6,Cold Laminator,ColdLaminatorV6,180,...,2010-03-04,2010-04-29,2010-03-22,Kong Yibo,Zhao Rongchun,11410306,Male,26,15074922863,NaN
49997,CSE0004999,State Power Corporation,Northern China,China,Beijing,Electricity,P4052D1,Photo Box,PhotoBox60,820,...,2010-03-04,2010-03-17,2010-04-15,Kong Yibo,Jiang Yuqiao,11411917,Female,26,15045639100,NaN
49998,CSE0004999,State Power Corporation,Northern China,China,Beijing,Electricity,O032T69,Online Telephone,OnlineTelephoneO8,870,...,2010-03-04,2010-03-04,2010-03-30,Kong Yibo,Lv Ruoke,12111718,Male,19,13949123975,NaN


In [29]:
np.array(big_table[{"supply center","city"}])

array([['Eastern China', 'Shanghai'],
       ['Eastern China', 'Shanghai'],
       ['Eastern China', 'Shanghai'],
       ...,
       ['Northern China', 'Beijing'],
       ['Northern China', 'Beijing'],
       ['Northern China', 'Beijing']], dtype=object)

In [12]:
##去重字典
dict_supply_center={}
dict_client_enterprise={}
dict_contract={}
dict_salesman={}
dict_product={}
dict_model={}
dict_order_table={}

In [13]:
##使用美元符号防止sql注入
size=big_table["contract number"].size
cnt=0
start=time.mktime(time.localtime())
for i in range(size):
    cnt=cnt+1
    if dict_supply_center.get(big_table["supply center"][i])==None:
        dict_supply_center[big_table["supply center"][i]]=1
        cur.execute("insert into supply_center(supply_center) "
               "values($$%s$$)"%(big_table["supply center"][i]))
    if dict_client_enterprise.get(big_table["client enterprise"][i])==None:
        dict_client_enterprise[big_table["client enterprise"][i]]=1
        cur.execute("insert into client_enterprise(client_enterprise,supply_center,country,city,industry) "
               "values($$%s$$,$$%s$$,$$%s$$,$$%s$$,$$%s$$)"
                    %(big_table["client enterprise"][i],big_table["supply center"][i],big_table["country"][i],big_table["city"][i],big_table["industry"][i])) 
    if dict_contract.get(big_table["contract number"][i])==None:
        dict_contract[big_table["contract number"][i]]=1
        cur.execute("insert into contract(contract_number,contract_date,director,client_enterprise)"
               "values($$%s$$,$$%s$$,$$%s$$,$$%s$$)"
                    %(big_table["contract number"][i],big_table["contract date"][i],big_table["director"][i],big_table["client enterprise"][i]))
    if dict_salesman.get(big_table["salesman number"][i])==None:
        dict_salesman[big_table["salesman number"][i]]=1
        cur.execute("insert into salesman(salesman_number,salesman_name,gender,age,mobile_phone,supply_center) "
               "values($$%s$$,$$%s$$,$$%s$$,%d,$$%s$$,$$%s$$)"
                    %(big_table["salesman number"][i],big_table["salesman"][i],big_table["gender"][i],big_table["age"][i],big_table["mobile phone"][i],big_table["supply center"][i])) 
    if dict_product.get(big_table["product code"][i])==None:
        dict_product[big_table["product code"][i]]=1
        cur.execute("insert into product(product_code,product_name) "
               "values($$%s$$,$$%s$$)"
                    %(big_table["product code"][i],big_table["product name"][i]))   
    if dict_model.get(big_table["product model"][i])==None:
        dict_model[big_table["product model"][i]]=1
        cur.execute("insert into model(product_model,unit_price,product_code) "
               "values($$%s$$,%d,$$%s$$)"
                    %(big_table["product model"][i],big_table["unit price"][i],big_table["product code"][i]))
    ##ps:order_table不需要字典去重，因为我们设置的是自增    
    cur.execute("insert into order_table(product_code,product_model,quantity,salesman_number,estimated_date,lodgement_date,contract_number) "
            "values($$%s$$,$$%s$$,%d,$$%s$$,$$%s$$,$$%s$$,$$%s$$)"
                %(big_table["product code"][i],big_table["product model"][i],big_table["quantity"][i],big_table["salesman number"][i],big_table["estimated delivery date"][i],big_table["lodgement date"][i],big_table["contract number"][i]))
con.commit()
con.close()
end=time.mktime(time.localtime())
print("Loading speed:"+str(cnt/(end-start))+" records/s")

Loading speed:4545.454545454545 records/s
